# 能力分类

## 模拟数据生成 

## 贝叶斯网络 

### 手工构建网络 （离散数据）

#### 建网络 

In [11]:
# 学生等级、学生成绩分级分成不同区段的离散标准
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD

# Defining the model structure. We can define the network by just passing a list of edges.
model = BayesianModel([('M', 'Q1'), ('I', 'Q2'), ('B', 'Q1'), ('B', 'Q2')])

cpd_m = TabularCPD(variable='M', variable_card=2, values=[[0.6, 0.4]],state_names={'M':['Low','High']})
cpd_i = TabularCPD(variable='I', variable_card=2, values=[[0.7, 0.3]],state_names={'I':['Low','High']})
cpd_b = TabularCPD(variable='B', variable_card=2, values=[[0.8, 0.2]],state_names={'B':['Low','High']})
# Q1 测试一，结果 0, 1, 2 (差，中，优)
cpd_q1 = TabularCPD(variable='Q1', variable_card=3, 
                   values=[[0.3, 0.05, 0.9,  0.5],
                           [0.4, 0.25, 0.08, 0.3],
                           [0.3, 0.7,  0.02, 0.2]],
                  evidence=['M', 'B'],
                  evidence_card=[2, 2],state_names={'Q1': ['A', 'B', 'C'],
                                   'M': ['Low', 'High'],
                                   'B': ['Low', 'High']})

cpd_q2 = TabularCPD(variable='Q2', variable_card=3, 
                   values=[[0.3, 0.05, 0.9,  0.5],
                           [0.4, 0.25, 0.08, 0.3],
                           [0.3, 0.7,  0.02, 0.2]],
                  evidence=['I', 'B'],
                  evidence_card=[2, 2],state_names={'Q2': ['A', 'B', 'C'],
                                   'I': ['Low', 'High'],
                                   'B': ['Low', 'High']})

model.add_cpds(cpd_m,  cpd_i, cpd_b, cpd_q1, cpd_q2)
model.check_model()
print(model.get_cpds())
print(cpd_q1)

[<TabularCPD representing P(M:2) at 0x221fdd17ec8>, <TabularCPD representing P(I:2) at 0x221fdd17e88>, <TabularCPD representing P(B:2) at 0x221fdd17fc8>, <TabularCPD representing P(Q1:3 | M:2, B:2) at 0x221fdd17288>, <TabularCPD representing P(Q2:3 | I:2, B:2) at 0x221fdd17bc8>]
+-------+--------+---------+---------+---------+
| M     | M(Low) | M(Low)  | M(High) | M(High) |
+-------+--------+---------+---------+---------+
| B     | B(Low) | B(High) | B(Low)  | B(High) |
+-------+--------+---------+---------+---------+
| Q1(A) | 0.3    | 0.05    | 0.9     | 0.5     |
+-------+--------+---------+---------+---------+
| Q1(B) | 0.4    | 0.25    | 0.08    | 0.3     |
+-------+--------+---------+---------+---------+
| Q1(C) | 0.3    | 0.7     | 0.02    | 0.2     |
+-------+--------+---------+---------+---------+


#### 推断 

In [29]:
# Getting all the local independencies in the network.
# Variable Elimination  消元, 对于有向的贝叶斯网络来说，每个节点联合其父节点为一个因子，当对某一变量进行消除时，把与该变量有关的
# 因子放到对应的连加符号右边，其他的变量与连加符号放到该连加符号左边。将所需消除的连加符号以及连加符号右边的式子，写成以右边因子中所
# 存在的所有变量为自变量的函数τ。　　注意，该步骤之后所有因式的辖域中均不存在被消除的变量。　　马尔可夫网的变量消除法与上述方法相同。 
from pgmpy.inference import VariableElimination
print('independ --> ',  model.local_independencies(['M', 'I', 'B', 'Q1', 'Q2']))
print('trail --> ', model.active_trail_nodes('Q1'))
# 对模型进行消元，简便模型运算
infer = VariableElimination(model)
print(infer.query(['Q1']))
print(infer.query(['Q1'], evidence={'M': 'Low', 'I': 'Low'}))  # I与Q1无关，这里只会计算M为Low情况下在的状态 

independ -->  (M _|_ I, Q2, B)
(I _|_ B, Q1, M)
(B _|_ I, M)
(Q1 _|_ I, Q2 | B, M)
(Q2 _|_ Q1, M | I, B)
trail -->  {'Q1': {'Q2', 'B', 'M', 'Q1'}}


Eliminating: M: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 285.85it/s]


+-------+-----------+
| Q1    |   phi(Q1) |
+=======+===========+
| Q1(A) |    0.4780 |
+-------+-----------+
| Q1(B) |    0.2716 |
+-------+-----------+
| Q1(C) |    0.2504 |
+-------+-----------+


Eliminating: B: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 250.14it/s]


+-------+-----------+
| Q1    |   phi(Q1) |
+=======+===========+
| Q1(A) |    0.2500 |
+-------+-----------+
| Q1(B) |    0.3700 |
+-------+-----------+
| Q1(C) |    0.3800 |
+-------+-----------+


### 手工构建网络（连续数据）

#### 生成模拟数据 

In [20]:
'''
模拟数据格式： id, 分数, 时间
网络结果： P(grade, time|I,B)
'''
import numpy as np
import scipy.stats as stats
size = 200
time_low, time_upper = 5, 45
mu, sigma = 25,10
exam_time = np.array(stats.truncnorm((time_low - mu) / sigma, (time_upper - mu) / sigma, loc=mu, scale=sigma).rvs(size), 
                     dtype=np.int)

low, upper = 10, 100
mu, sigma = 85,10
score = np.array(stats.truncnorm((low - mu) / sigma, (upper - mu) / sigma, loc=mu, scale=sigma).rvs(size), 
                     dtype=np.int)
# score = np.random.normal(50,0.5, number).astype(np.float)
# print('etime -->', etime)
idnum = np.array(range(size))
record = list(zip(idnum, score, exam_time))
np.savetxt('D:\\record.txt',record,fmt='%d',delimiter=',')

#### 建网络 

In [ ]:
# https://blog.csdn.net/wzgbm/article/details/51680540 概率中的pdf
